In [1]:
# %%
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--model_name', type=str, default='InceptionV3')
parser.add_argument('--weights', type=str, default='radimagenet')
parser.add_argument('--n_layers', type=int, default=4)
parser.add_argument('--n_neurons', type=int, default=256)
parser.add_argument('--n_dropout', type=float, default=0.0)
parser.add_argument('--lr_1', type=float, default=3e-4)
parser.add_argument('--lr_2', type=float, default=3e-6)
parser.add_argument('--image_size', type=int, default=512, required=False)
parser.add_argument('--batch_size', type=int, default=16, required=False)

args = parser.parse_args("")

model_name = args.model_name
weights = args.weights
n_layers = args.n_layers
n_neurons = args.n_neurons
n_dropout = args.n_dropout
lr_1 = args.lr_1
lr_2 = args.lr_2
img_size = args.image_size
batch_size = args.batch_size


In [2]:
# %%
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import matplotlib.pylab as plt
from sklearn.metrics import roc_curve, roc_auc_score, recall_score
from tensorflow.keras.applications import InceptionResNetV2, ResNet50, InceptionV3, DenseNet121, Xception
import tensorflow_hub as hub
import pandas as pd
from sklearn.metrics import roc_auc_score, accuracy_score
import keras_cv


# %%
train_dir = '../data/split_1/train'
val_dir = '../data/split_1/val'
test_dir = '../data/split_1/test'


# %%
train_ds = tf.keras.preprocessing.image_dataset_from_directory(train_dir, label_mode='binary', seed=0, image_size=(img_size, img_size), batch_size=batch_size, color_mode='rgb')
val_ds = tf.keras.preprocessing.image_dataset_from_directory(val_dir, label_mode='binary', seed=0, image_size=(img_size, img_size), batch_size=batch_size, color_mode='rgb')
test_ds = tf.keras.preprocessing.image_dataset_from_directory(test_dir, label_mode='binary', seed=0, image_size=(img_size, img_size), batch_size=1, color_mode='rgb')


# %%
#Apply data augmentation
preprocessing_model = tf.keras.Sequential()
preprocessing_model.add(
    tf.keras.layers.experimental.preprocessing.RandomRotation(40))
preprocessing_model.add(
    tf.keras.layers.experimental.preprocessing.RandomTranslation(0.2, 0.2))
preprocessing_model.add(
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2, 0.2))
preprocessing_model.add(
    tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal"))
preprocessing_model.add(
    tf.keras.layers.experimental.preprocessing.RandomFlip(mode="vertical"))


# %%
train_ds = train_ds.map(lambda images, labels:
                        (preprocessing_model(images), labels))


SyntaxError: invalid syntax (1925908409.py, line 41)

In [3]:
if model_name == 'InceptionResNetV2':
    preprocess_fx = tf.keras.applications.inception_resnet_v2.preprocess_input
    model_dir = "../RadImageNet/models/RadImageNet-IRV2_notop.h5"
    if weights == 'imagenet':
        base_model = InceptionResNetV2(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet', pooling='avg')
    elif weights == 'radimagenet':
        base_model = InceptionResNetV2(input_shape=(img_size, img_size, 3), include_top=False, weights=model_dir, pooling='avg')
elif model_name == 'ResNet50':
    preprocess_fx = tf.keras.applications.resnet50.preprocess_input
    model_dir = "../RadImageNet/models/RadImageNet-ResNet50_notop.h5"
    if weights == 'imagenet':
        base_model = ResNet50(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet', pooling='avg')
    elif weights == 'radimagenet':
        base_model = ResNet50(input_shape=(img_size, img_size, 3), include_top=False, weights=model_dir, pooling='avg')
elif model_name == 'InceptionV3':
    preprocess_fx = tf.keras.applications.inception_v3.preprocess_input
    model_dir = "../RadImageNet/models/RadImageNet-InceptionV3_notop.h5"
    if weights == 'imagenet':
        base_model = InceptionV3(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet', pooling='avg')
    elif weights == 'radimagenet':
        base_model = InceptionV3(input_shape=(img_size, img_size, 3), include_top=False, weights=model_dir, pooling='avg')
elif model_name == 'DenseNet121':
    preprocess_fx = tf.keras.applications.densenet.preprocess_input
    model_dir = "../RadImageNet/models/RadImageNet-DenseNet121_notop.h5"
    if weights == 'imagenet':
        base_model = DenseNet121(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet', pooling='avg')
    elif weights == 'radimagenet':
        base_model = DenseNet121(input_shape=(img_size, img_size, 3), include_top=False, weights=model_dir, pooling='avg')
elif model_name == 'Xception':
    preprocess_fx = tf.keras.applications.xception.preprocess_input
    if weights == 'imagenet':
        base_model = tf.keras.applications.Xception(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet', pooling='avg')
elif model_name == 'BiT':
    base_model = hub.KerasLayer("https://tfhub.dev/google/bit/m-r50x1/1", trainable=False)
    preprocess_fx = tf.keras.applications.resnet50.preprocess_input
    


In [4]:


inputs = keras.Input(shape=(img_size, img_size, 3))
y = preprocess_fx(inputs)
y = base_model(y, training=False)
for i in range(n_layers):
    y = keras.layers.Dense(n_neurons, activation='relu')(y)
    y = keras.layers.Dropout(n_dropout)(y)
outputs = keras.layers.Dense(1, activation='sigmoid')(y)
model = keras.Model(inputs, outputs)

# %%
early_stopping = keras.callbacks.EarlyStopping(patience=100, min_delta=1e-10, restore_best_weights=True)

# %%

model.compile(
    optimizer=keras.optimizers.Adam(lr_1), 
    loss=keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.AUC()],
)

epochs = 1000


In [5]:
model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=early_stopping, verbose=1)
print('phase 1 complete')
# %%
#unfreeze all layers and train at lower learning rate
base_model.trainable = True
model.compile(
    optimizer=keras.optimizers.Adam(lr_2), 
    loss=keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.AUC()],
)
model.fit(train_ds, epochs=1000, validation_data=val_ds, callbacks=early_stopping, verbose=1)
print('phase 2 complete')


Epoch 1/1000


2023-02-13 17:56:49.332132: E tensorflow/stream_executor/cuda/cuda_dnn.cc:371] Could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
2023-02-13 17:56:49.332403: E tensorflow/stream_executor/cuda/cuda_dnn.cc:379] Possibly insufficient driver version: 525.85.12
2023-02-13 17:56:49.332450: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1120 : UNKNOWN: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.


UnknownError: 2 root error(s) found.
  (0) UNKNOWN:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/inception_v3/conv2d/Conv2D
 (defined at /home/kchen/.local/lib/python3.8/site-packages/keras/layers/convolutional.py:231)
]]
	 [[assert_less_equal/Assert/AssertGuard/pivot_f/_13/_51]]
  (1) UNKNOWN:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/inception_v3/conv2d/Conv2D
 (defined at /home/kchen/.local/lib/python3.8/site-packages/keras/layers/convolutional.py:231)
]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_19999]

Errors may have originated from an input operation.
Input Source operations connected to node model/inception_v3/conv2d/Conv2D:
In[0] model/tf.math.subtract/Sub (defined at /home/kchen/.loca

In [ ]:
# # %%
# #load test set and normalize using same code

# y_val_pred = np.array([])
# y_val = np.array([])
# for images, labels in val_ds:
#     y_val = np.append(y_val, labels.numpy())
#     y_val_pred = np.append(y_val_pred, model.predict(images))
# auroc_val = roc_auc_score(y_val, y_val_pred)

# y_test_pred = np.array([])
# y_test = np.array([])
# for images, labels in test_ds:
#     y_test = np.append(y_test, labels.numpy())
#     y_test_pred = np.append(y_test_pred, model.predict(images))
# auroc_test = roc_auc_score(y_test, y_test_pred)
# file_paths = test_ds.file_paths
# pred_df = pd.DataFrame({'file_path': file_paths, 'y_true': y_test, 'y_pred': y_test_pred})
# pred_df.to_csv('preds/preds_{}_{}.csv'.format(model_name, weights), index=False)

In [ ]:
#save the final model as a .h5 file
model.save('models/{}_{}.h5'.format(model_name, weights))

In [ ]:
#generate predictions for all images in the dataset, including train and val, and save to one csv
train_ds = tf.keras.preprocessing.image_dataset_from_directory(train_dir, label_mode='binary', seed=0, shuffle=False, image_size=(img_size, img_size), batch_size=batch_size, color_mode='rgb')
val_ds = tf.keras.preprocessing.image_dataset_from_directory(val_dir, label_mode='binary', seed=0, shuffle=False, image_size=(img_size, img_size), batch_size=batch_size, color_mode='rgb')
test_ds = tf.keras.preprocessing.image_dataset_from_directory(test_dir, label_mode='binary', seed=0, shuffle=False, image_size=(img_size, img_size), batch_size=1, color_mode='rgb')
y_train_pred = np.array([])
y_train = np.array([])
for images, labels in train_ds:
    y_train = np.append(y_train, labels.numpy())
    y_train_pred = np.append(y_train_pred, model.predict(images))
train_file_paths = train_ds.file_paths

y_val_pred = np.array([])
y_val = np.array([])
for images, labels in val_ds:
    y_val = np.append(y_val, labels.numpy())
    y_val_pred = np.append(y_val_pred, model.predict(images))
val_file_paths = val_ds.file_paths

y_test_pred = np.array([])
y_test = np.array([])
for images, labels in test_ds:
    y_test = np.append(y_test, labels.numpy())
    y_test_pred = np.append(y_test_pred, model.predict(images))
test_file_paths = test_ds.file_paths

y_all_pred = np.append(y_train_pred, y_val_pred)
y_all_pred = np.append(y_all_pred, y_test_pred)
y_all = np.append(y_train, y_val)
y_all = np.append(y_all, y_test)
file_paths = np.append(train_file_paths, val_file_paths)
file_paths = np.append(file_paths, test_file_paths)
pred_df = pd.DataFrame({'file_path': file_paths, 'y_true': y_all, 'y_pred': y_all_pred})
pred_df.to_csv('preds/preds_{}_{}_all_v2.csv'.format(model_name, weights), index=False)


Found 492 files belonging to 2 classes.
Found 165 files belonging to 2 classes.
Found 164 files belonging to 2 classes.


In [ ]:
auroc_test = roc_auc_score(y_test, y_test_pred)
auroc_val = roc_auc_score(y_val, y_val_pred)

results_df = pd.DataFrame({'model': [model_name], 'weights': [weights], 'image_size': [img_size], 'batch_size': [batch_size], 'lr_1': [lr_1], 'lr_2': [lr_2], 'n_layers': [n_layers], 'n_neurons': [n_neurons], 'n_dropout': [n_dropout], 'auroc_val': auroc_val, 'auroc_test': auroc_test})

In [ ]:
print("test auroc: {}".format(auroc_test))
print("val auroc: {}".format(auroc_val))

test auroc: 0.7014013118664282
val auroc: 0.659888004715591


In [ ]:
#if results file exists, append to it, otherwise create it
# try:
#     results_df.to_csv('results/results_iv3.csv', mode='a', header=False, index=True)
# except:
#     results_df.to_csv('results/results_iv3.csv', header=True, index=True)